In [1]:
# ! conda install -c conda-forge spacy
# ! python -m spacy download en_core_web_sm

In [55]:
import spacy 
import nltk
from itertools import product
import csv, sqlite3

nlp = spacy.load('en_core_web_sm') 

In [51]:
# sentence = "What are total death cases in US on 27-09-2020?"
# sentence = "number of cities with total cases greater than 1000?"

def get_entities(sentence):
	doc = nlp(sentence)
	print("----> Entities:")
	for ent in doc.ents: 
		print("-------->",ent.text, ent.start_char, ent.end_char, ent.label_) 

def get_tokens(sentence):
	doc = nlp(sentence) 
	print("----> Tokens:")
	for token in doc: 
		# if(token.dep_ == "nsubj"):
		print("-------->", token.text, token.pos_, token.dep_) 

In [52]:
queries=[]
with open("../possible-questions.txt","r") as f:
    for line in f.readlines():
        line = line[:-1]
        if(line):
            queries.append(line)

In [54]:
for sent in queries:
    print(sent)
    get_entities(sent)
    get_tokens(sent)
    doc=nlp(sent)
    print()


total number of cases found in Afganistan?
----> Entities:
--------> Afganistan 31 41 GPE
----> Tokens:
--------> total ADJ amod
--------> number NOUN ROOT
--------> of ADP prep
--------> cases NOUN pobj
--------> found VERB acl
--------> in ADP prep
--------> Afganistan PROPN pobj
--------> ? PUNCT punct

total number of cases found in Colombia till july?
----> Entities:
--------> Colombia 31 39 GPE
--------> july 45 49 DATE
----> Tokens:
--------> total ADJ amod
--------> number NOUN ROOT
--------> of ADP prep
--------> cases NOUN pobj
--------> found VERB acl
--------> in ADP prep
--------> Colombia PROPN pobj
--------> till SCONJ prep
--------> july PROPN pobj
--------> ? PUNCT punct

total number of new cases found in France in april?
----> Entities:
--------> France 35 41 GPE
--------> april 45 50 DATE
----> Tokens:
--------> total ADJ amod
--------> number NOUN ROOT
--------> of ADP prep
--------> new ADJ amod
--------> cases NOUN pobj
--------> found VERB acl
--------> in ADP p

In [50]:
from nltk.corpus import wordnet

synonyms = []

for syn in wordnet.synsets("recovered"): 
    print(syn.name())
    print(syn.lemmas())
    for l in syn.lemmas(): 
        synonyms.append(l.name())
print (set(synonyms))

recover.v.01
[Lemma('recover.v.01.recover'), Lemma('recover.v.01.retrieve'), Lemma('recover.v.01.find'), Lemma('recover.v.01.regain')]
recuperate.v.04
[Lemma('recuperate.v.04.recuperate'), Lemma('recuperate.v.04.recover'), Lemma('recuperate.v.04.convalesce')]
recover.v.03
[Lemma('recover.v.03.recover'), Lemma('recover.v.03.go_back'), Lemma('recover.v.03.recuperate')]
recover.v.04
[Lemma('recover.v.04.recover'), Lemma('recover.v.04.recoup'), Lemma('recover.v.04.recuperate')]
reclaim.v.02
[Lemma('reclaim.v.02.reclaim'), Lemma('reclaim.v.02.recover')]
recover.v.06
[Lemma('recover.v.06.recover')]
cured.s.01
[Lemma('cured.s.01.cured'), Lemma('cured.s.01.healed'), Lemma('cured.s.01.recovered')]
recovered.s.02
[Lemma('recovered.s.02.recovered')]
{'recuperate', 'healed', 'find', 'recoup', 'reclaim', 'regain', 'convalesce', 'go_back', 'recovered', 'cured', 'retrieve', 'recover'}


In [73]:
from nltk.corpus import wordnet 

def caseTypeProbability(sent):
    docs=nlp(sent)
    case_type=["death", "recovery", "active", "confirm"]

    similarity=0.0
    type="total"
    word=""

    # for case in case_type:
    #     tempdoc=nlp(case)
    #     tempdoc=tempdoc[0]
        
    #     for doc in docs:
    #         temp=doc.similarity(tempdoc)

    #         if similarity<temp:
    #             similarity=temp
    #             type=case
    #             word=doc
    
    for case in case_type:
        w1=wordnet.synsets(case)
        # print(w1)
        for doc in docs:
            w2=wordnet.synsets(str(doc.text))
            # print(w2)
            # temp=w1.similarity(w2)

            for i,j in list(product(*[w1,w2])):
                # print(i,j)
                temp = i.wup_similarity(j) # Wu-Palmer Similarity
                # maxscore = score if maxscore < score else maxscore
                if temp and similarity<temp:
                    similarity=temp
                    type=case
                    word=doc
    
    return type, word, similarity

In [74]:
for query in queries:
    print(query)
    print(caseTypeProbability(query))

total number of cases found in Afganistan?
('death', cases, 0.7692307692307693)
total number of cases found in Colombia till july?
('death', cases, 0.7692307692307693)
total number of new cases found in France in april?
('death', cases, 0.7692307692307693)
total number of new cases found in Greece between april to september?
('death', cases, 0.7692307692307693)
total number cases recovered in Hungary?
('death', cases, 0.7692307692307693)
total number of cases recovered in Iceland till May?
('death', cases, 0.7692307692307693)
total number of new cases recovered in Indonesia in May?
('death', cases, 0.7692307692307693)
total number of new cases recovered in india from March to December?
('death', cases, 0.7692307692307693)
total number of deaths in india?
('death', deaths, 1.0)
total number of deaths in china till august?
('death', deaths, 1.0)
total number of deaths in US in august?
('death', deaths, 1.0)
total number of deaths in poland between august and december?
('death', deaths, 1

In [9]:
from spacy.kb import KnowledgeBase

# nlp = spacy.load('en_core_web_sm')
kb = KnowledgeBase(vocab=nlp.vocab, entity_vector_length=3)

# adding entities
# kb.add_entity(entity="Q1004791", freq=6, entity_vector=[0, 3, 5])
# kb.add_entity(entity="Q42", freq=342, entity_vector=[1, 9, -3])
# kb.add_entity(entity="Q5301561", freq=12, entity_vector=[-2, 4, 2])

# # adding aliases
# kb.add_alias(alias="Douglas", entities=["Q1004791", "Q42", "Q5301561"], probabilities=[0.6, 0.1, 0.2])

print("Number of entities in KB:",kb.get_size_entities()) # 3
print("Number of aliases in KB:", kb.get_size_aliases()) # 2

candidates = kb.get_candidates("Douglas")
for c in candidates:
    print(" ", c.entity_, c.prior_prob, c.entity_vector)

Number of entities in KB: 0
Number of aliases in KB: 0


In [165]:
def csv_to_table(path, csv_name, table_name):
    with open(dataset_path+csv_name,'r') as fin:
        dr = csv.DictReader(fin)
        to_db=[tuple(i.values()) for i in dr]
    
    count=len(dr.fieldnames)
    bindings="?, "*count

    cur.executemany("INSERT INTO "+table_name+" VALUES ("+bindings[:-2]+");", to_db)


In [176]:
# def create_database_schema():
con = sqlite3.connect(r"D:\My Study Folder\Others\NLP-Search-Engine-COVID-19-Dataset\dataset\covid-19\mysql_database\covid19.db")
cur = con.cursor()

dataset_path="../dataset/covid-19/required_only/"

tables=[("worldwide_aggregate"),("reference"),("timeseries"),("us")]
for table in tables:
    cur.execute("DROP TABLE IF EXISTS "+table+";")

cur.execute("create table worldwide_aggregate(Date Date NOT NULL, Confirmed BIGINT NOT NULL, Recovered BIGINT NOT NULL, Deaths BIGINT NOT NULL, Increase_rate FLOAT default NULL);")

csv_to_table(dataset_path,"worldwide-aggregate.csv", tables[0])

cur.execute("create table us(Date Date NOT NULL, Admin2 VARCHAR(100) NOT NULL, Province_State VARCHAR(100) NOT NULL, Confirmed BIGINT NOT NULL, Deaths BIGINT NOT NULL, Country_Region VARCHAR(100) NOT NULL);")

csv_to_table(dataset_path,"us_simplified.csv", tables[3])

cur.execute("create table reference(UID INT NOT NULL, iso2 VARCHAR(20), iso3 VARCHAR(20), code3 INT, FIPS INT, Admin2 VARCHAR(100) NOT NULL, Province_State VARCHAR(100) NOT NULL, Country_Region VARCHAR(100) NOT NULL, Lat FLOAT NOT NULL, Long_ FLOAT NOT NULL, Combined_Key VARCHAR(100), Popolation BIGINT NOT NULL);")

csv_to_table(dataset_path,"reference.csv", tables[1])

cur.execute("create table timeseries(Date Date NOT NULL, Country_Region VARCHAR(100) NOT NULL, Province_State VARCHAR(100), Confirmed BIGINT NOT NULL, Recovered BIGINT NOT NULL, Deaths BIGINT NOT NULL);")

csv_to_table(dataset_path,"time-series-19-covid-combined.csv", tables[2])

con.commit()

with open('covid19-schema.sql','w') as fp:
    for line in con.iterdump():
        fp.write('%s\n' % line)

con.close()


# create_database_schema()